# json brut à CSV
### Création CSV to_eval, liste de IPC secondaire pour ES

In [93]:
import os
os.chdir('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP')
print(os.getcwd())

C:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP


In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm


In [96]:
mypath = "data/2006/"  #'data/2016/2016/'
_2006_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [22]:
print(len(_2006_files))
print(_2006_files[1])
patent_path = mypath + _2006_files[9]
print(patent_path)
with open(patent_path) as f:
    d = json.load(f) # load json in d
    f.close()   # close f - not needed with with ___ as syntax
class_mainIPC = d['main_ipcr_label']
print(d)
print(d["main_ipcr_label"])

for i in tqdm(range(10)):
    patent_path = mypath + _2006_files[i]
    with open(patent_path) as f:
        d = json.load(f) # load json in d
        f.close()   # close f - not needed with with ___ as syntax
    class_mainIPC = d['main_ipcr_label']
    secClass = d['ipcr_labels']
    if len(secClass) > 1:
        print(i)

    


207936
10018639.json
C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2004/10149078.json
{'application_number': '10149078', 'publication_number': 'US20060175173A1-20060810', 'title': 'Force -transmision unit comprising speed -dependent hydraulic clutch and centrifugal force compensation', 'decision': 'ACCEPTED', 'date_produced': '20060726', 'date_published': '20060810', 'main_cpc_label': '', 'cpc_labels': [], 'main_ipcr_label': 'F16D2512', 'ipcr_labels': ['F16D2512'], 'patent_number': '7287635', 'filing_date': '20040114', 'patent_issue_date': '20071030', 'abandon_date': '', 'uspc_class': '192', 'uspc_subclass': '035000', 'examiner_id': '63138.0', 'examiner_name_last': 'RODRIGUEZ', 'examiner_name_first': 'SAUL', 'examiner_name_middle': '', 'inventor_list': [{'inventor_name_last': 'Gratzer', 'inventor_name_first': 'Franz', 'inventor_city': 'Stallhofen', 'inventor_state': '', 'inventor_country': 'AT'}], 'abstract': 'A power transmission unit with a hydraulic coupling dependent 

100%|██████████| 10/10 [00:00<00:00, 1150.39it/s]


Création de listes de brevets de chaque type de class IPC à préciser

In [29]:
# Check on all decision and repartition -- I think we should keep only the one accepted based on other USPTO patent (continuity)

# initialisation de dict, dont les valeurs inconnues sont initialiser à 0, pour chaque classe IPC
decision_H01L = collections.defaultdict(int)
decision_G06F = collections.defaultdict(int)
decision_A61B = collections.defaultdict(int)
nbEmpty = collections.defaultdict(int)

# initialisation de vecteurs pour chaque classe IPC
patent_H01L_list = []
patent_G06F_list = []
patent_A61B_list = []

for i in tqdm(range(len(_2006_files[:10000]))):
    patent_path = mypath + _2006_files[i]
    with open(patent_path) as f:
        d = json.load(f) # load json in d
        f.close()   # close f - not needed with "with ___ as" syntax
    class_mainIPC = d['main_ipcr_label']
    if re.match(r'^H01L', class_mainIPC):
        patent_H01L_list.append(_2006_files[i])
        decision_H01L[d['decision']] += 1
    if re.match(r'^G06F', class_mainIPC):
        patent_G06F_list.append(_2006_files[i])
        decision_G06F[d['decision']] += 1
    if re.match(r'^A61B', class_mainIPC):
        patent_A61B_list.append(_2006_files[i])
        decision_A61B[d['decision']] += 1
    if class_mainIPC == "":
        nbEmpty[class_mainIPC] += 1

100%|██████████| 10000/10000 [00:03<00:00, 3331.98it/s]


In [31]:
print(len(patent_H01L_list), len(patent_G06F_list), len(patent_A61B_list))
print(decision_H01L, decision_G06F, decision_A61B, nbEmpty)

252 390 191
defaultdict(<class 'int'>, {'ACCEPTED': 137, 'REJECTED': 115}) defaultdict(<class 'int'>, {'ACCEPTED': 210, 'REJECTED': 179, 'PENDING': 1}) defaultdict(<class 'int'>, {'REJECTED': 110, 'ACCEPTED': 80, 'PENDING': 1}) defaultdict(<class 'int'>, {'': 6})


Il y a énormément de brevets sans IPC...

Surtout en 2004, mais aussi _2006

- 10018639 - _20060081931A1
- 10018320 - _20060130116A1
- 10250708 - 20060086443A1

Nous allons continuer avec 2006 ici

#### Exclusion des de tout autre que Accepter ou rejeter, dans une liste de brevet de classe IPC

In [32]:
final_patents = []
decision = []
for i in tqdm(range(len(patent_A61B_list))):
    patent_path = mypath + patent_A61B_list[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    if d['decision'] == 'ACCEPTED' or d['decision'] == 'REJECTED': # exclure pending
        final_patents.append(patent_A61B_list[i])
        decision.append(d['decision'])

100%|██████████| 191/191 [00:00<00:00, 2999.42it/s]


In [34]:
print(final_patents)
print(decision)

['10484186.json', '10493926.json', '10505531.json', '10511495.json', '10512150.json', '10512772.json', '10513656.json', '10516287.json', '10518848.json', '10518867.json', '10519023.json', '10520965.json', '10521373.json', '10522197.json', '10522609.json', '10524196.json', '10524984.json', '10526112.json', '10527781.json', '10528417.json', '10528504.json', '10528841.json', '10528922.json', '10529150.json', '10529770.json', '10530094.json', '10530811.json', '10530860.json', '10531204.json', '10531378.json', '10531764.json', '10531819.json', '10531832.json', '10531958.json', '10532691.json', '10532704.json', '10533596.json', '10534268.json', '10535214.json', '10535227.json', '10535316.json', '10535617.json', '10535726.json', '10535730.json', '10535775.json', '10535914.json', '10535958.json', '10536480.json', '10536503.json', '10536605.json', '10537103.json', '10537235.json', '10537795.json', '10538155.json', '10538411.json', '10538992.json', '10539175.json', '10539255.json', '10539818.jso

#### Création CSV à partir de list de json (1 par classe IPC et par année)

Infos à extraire: \
- numéro de brevet
- informations textuelles :
    - Anciens
        - ID
        - titre
        - background
        - claims
        - Décision
    - Ajout:
        - Abstract (pourquoi on ajoute?)
        - Summary (pourquoi on ajoute?)
        - IPC principale (pour seconde analyse)
        - IPC secondaire (pour expectation space, pas directement dans CSV)


In [68]:
non_main_ipc = []
labels = []
patent_number, titles, backgrounds, claims, summary, abstract, ipc = [], [], [], [], [], [], []
for i in tqdm(range(len(final_patents))):
    patent_path = mypath + final_patents[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close() # ligne inutile
    
    #Creating the lists for the other information
    patent_number.append(d['application_number'])
    titles.append(d['title'])
    backgrounds.append(d['background'])
    claims.append(d['claims'])
    summary.append(d['summary'])
    abstract.append(d['abstract'])
    ipc.append(d['main_ipcr_label'])

    #Collecting non main ipc class
    non_main =  d['ipcr_labels']
    for ipcr in non_main:
        non_main_ipc.append(ipcr) # only 4 first characters to be sure of being at same level
    #Getting labels based on decision
    label = 0
    if d['decision'] == 'ACCEPTED':
        label = 1
    labels.append(label)

100%|██████████| 190/190 [00:00<00:00, 1451.21it/s]


only 4 first characters to be sure of being at same level \
Getting labels based on decision????

In [50]:
pd.options.display.max_colwidth
pd.options.display.max_rows

60

In [73]:
print(len(non_main_ipc))
print(patent_number)
print(non_main_ipc)
print(claims[0])
print(len(ipc))
print(len(labels))

253
['10484186', '10493926', '10505531', '10511495', '10512150', '10512772', '10513656', '10516287', '10518848', '10518867', '10519023', '10520965', '10521373', '10522197', '10522609', '10524196', '10524984', '10526112', '10527781', '10528417', '10528504', '10528841', '10528922', '10529150', '10529770', '10530094', '10530811', '10530860', '10531204', '10531378', '10531764', '10531819', '10531832', '10531958', '10532691', '10532704', '10533596', '10534268', '10535214', '10535227', '10535316', '10535617', '10535726', '10535730', '10535775', '10535914', '10535958', '10536480', '10536503', '10536605', '10537103', '10537235', '10537795', '10538155', '10538411', '10538992', '10539175', '10539255', '10539818', '10540778', '10540890', '10540912', '10540985', '10541123', '10541231', '10541719', '10541896', '10542350', '10542387', '10542600', '10543001', '10543400', '10543445', '10543477', '10543654', '10543739', '10543793', '10543992', '10544013', '10544015', '10544102', '10544243', '10544323',

In [54]:
expectations_classes = list(set(non_main_ipc)) # création d'éléments unique - 253 à 76 éléments
print(expectations_classes)
print(len(expectations_classes))

['A61B51455', 'A61B50402', 'A61B1760', 'A61B124', 'A61B5117', 'A61B5055', 'A61B1726', 'A61B1300', 'A61F224', 'A61L218', 'G01N2730', 'G01N340', 'A61B5151', 'A61B500', 'A61B603', 'A61B806', 'G01N2300', 'A61B1756', 'A61B1802', 'A61B17122', 'A61B508', 'A61B100', 'A61N104', 'A61B132', 'A61B800', 'A61B17128', 'G21K112', 'A62B700', 'B65B5502', 'A61B1818', 'A61B505', 'A61B1734', 'A61B1900', 'A61B1700', 'A61B310', 'A61B104', 'A61B314', 'B65D8310', 'A61B1804', 'A61B1704', 'A61B5103', 'A61B814', 'A61B1758', 'A61B1703', 'A61M2500', 'A61N139', 'A61M100', 'A61B1706', 'A61B300', 'A61B1720', 'G01N33487', 'B65D8500', 'A61M3700', 'A61B1814', 'A61B600', 'A61N100', 'A61B1708', 'A61B302', 'B60K2800', 'B65D8100', 'A61B1780', 'A61B504', 'A61L200', 'F16K3102', 'A61N506', 'H05G160', 'A61B173205', 'A61B1732', 'A61B1782', 'A61B1000', 'A61F537', 'A61B1902', 'A61M1600', 'A61B1750', 'A61B502', 'A61B1267']
76


In [75]:
# Retirer les classes secondaires de la même classe que la princiaple
good_expectations_classes = []
for ipcr in expectations_classes:
    if ipcr[0:4] != "A61B":
        good_expectations_classes.append(ipcr)
        
print(len(good_expectations_classes))
print(good_expectations_classes)

25
['A61F224', 'A61L218', 'G01N2730', 'G01N340', 'G01N2300', 'A61N104', 'G21K112', 'A62B700', 'B65B5502', 'B65D8310', 'A61M2500', 'A61N139', 'A61M100', 'G01N33487', 'B65D8500', 'A61M3700', 'A61N100', 'B60K2800', 'B65D8100', 'A61L200', 'F16K3102', 'A61N506', 'H05G160', 'A61F537', 'A61M1600']


In [97]:
# Sauvegarder IPC secondaire en format texte
with open(r'exemple données/expectation_space/2006_A61B_expectation_IPC_class.txt', 'w') as fp:
    for item in good_expectations_classes:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [103]:
# Création jdD à mettre en csv
df = pd.DataFrame({'application_number': patent_number, 'title': titles, 'abstract':abstract,
                    'claims':claims, 'background': backgrounds, 'summary':summary, 'ipc':ipc, 'label': labels})
df

,application_number,title,abstract,claims,background,summary,ipc,label
0,10484186,Clip and method for epicardial placement of te...,The present invention provides for a sutureles...,1. A sutureless clip apparatus for attaching a...,<SOH> BACKGROUND OF THE INVENTION <EOH>1. Fie...,<SOH> SUMMARY OF THE INVENTION <EOH>Briefly d...,A61B504,0
1,10493926,Instrument for creating space in a human phary...,Instrument (700) which is to be introduced int...,1. Instrument (1) which is to be introduced in...,,,A61B100,0
2,10505531,Soft tissue anchor,The present invention relates to an apparatus ...,1. A bone screw for anchoring a suture thread ...,<SOH> BACKGROUND OF THE INVENTION <EOH>In the...,<SOH> BRIEF DESCRIPTIONS OF THE DRAWINGS <EOH...,A61B1704,0
3,10511495,System and method for piercing dermal tissue,A system for piercing dermal tissue includes a...,"1. A system for piercing dermal tissue, the sy...",<SOH> BACKGROUND OF INVENTION <EOH>1. Field o...,<SOH> SUMMARY OF INVENTION <EOH>Embodiments o...,A61B505,0
4,10512150,Apparatus for insertion of a medical device du...,The end-effector (150) includes a sheath (152)...,1. An interventional device for use while a su...,<SOH> BACKGROUND OF THE INVENTION <EOH>Prosta...,<SOH> SUMMARY OF THE INVENTION <EOH>The prese...,A61B505,1
...,...,...,...,...,...,...,...,...
185,10560263,Method and apparatus for body fluid sampling a...,The method comprises providing a lancing devic...,1-6. (canceled) 7. A device for body fluid sam...,<SOH> BACKGROUND OF THE INVENTION <EOH>Lancin...,<SOH> SUMMARY OF THE INVENTION <EOH>The prese...,A61B1732,0
186,10560265,Electrosurgical devices and methods for select...,Methods and devices for cutting or coagulating...,"1. A device for cutting or coagulating tissue,...",<SOH> BACKGROUND OF THE INVENTION <EOH>There ...,<SOH> SUMMARY OF THE INVENTION <EOH>The prese...,A61B1814,1
187,10560266,Device and methods useable for treatment of gl...,A device and method for cutting or ablating ti...,1. A device for cutting or ablating tissue in ...,<SOH> BACKGROUND OF THE INVENTION <EOH>A. Sym...,<SOH> SUMMARY OF THE INVENTION <EOH>The prese...,A61B1814,0
188,10560267,Tubular Cutter Device And Methods For Cutting ...,Methods and devices for cutting strips of tiss...,1. A device for cutting a strip of tissue of a...,<SOH> BACKGROUND OF THE INVENTION <EOH>There ...,<SOH> SUMMARY OF THE INVENTION <EOH>The prese...,A61B1732,1


In [105]:
# Sauvegarder CSV to_eval
df.to_csv('exemple données/to_eval/2014_A61B_patents_toeval.csv', index=False)